<a href="https://colab.research.google.com/github/SMKIDRaadet/SMKID-Point/blob/main/SMKID_Point.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#S/M-KID Point

Hvis du ønsker at indtaste S/M-KID Point, så foregår det [her](https://docs.google.com/spreadsheets/d/17TpTYWzlw-e8gREHyEGf-x-6guDb8R1yz_ObyLPLRPo/edit?usp=sharinghttps://).



# Under the hood

In [1]:
current_year = '2020/2021'

meeting_attendance = {'14/09/2020' : 'https://docs.google.com/spreadsheets/d/16PDwASOuUfmcC4sFNirVQSvVtK3tII8crOSLvxCY0Lo',
                      '28/09/2020' : 'https://docs.google.com/spreadsheets/d/1kq55taNGA8W2JU_Qbnjn0NVrwSR3yORsXvp9-8qFfR4'}

In [2]:
#Loading dependencies
import pandas as pd
import numpy as np
import os
from IPython.display import display, HTML

Derudover skal vi også lige have mountet Google Drive, så den kan hente deltagerlisterne og indtastningsarket. 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

if 'S M-KID Rådet' not in os.listdir('/content/drive/My Drive/'):
  raise ValueError('S/M-KID Rådets folder not in your Google Drive.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#Load the indtastningssheet.
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/17TpTYWzlw-e8gREHyEGf-x-6guDb8R1yz_ObyLPLRPo/')

point = pd.DataFrame(sh.worksheet(current_year).get_all_values())
point.columns = point.iloc[0]
point = point[1:]

#Create the dataframe for the added points.
point['Point'] = point['Point'].replace({',' : '.'}, regex=True).astype(float)

#Load all of the meetings

dates                 = np.asarray([])
attendees_name        = np.asarray([])
attendees_studynumber = np.asarray([])
activity              = np.asarray([])

for date in meeting_attendance:
  #Load the attendees.
  df_temp = pd.DataFrame(gc.open_by_url(meeting_attendance[date]).get_worksheet(0).get_all_values())
  df_temp.columns = df_temp.iloc[0]
  df_temp = df_temp[1:]

  studynumbers = df_temp['Studienummer'].to_numpy()
  names = df_temp['Fulde navn'].to_numpy()

  #Remove unique studynumbers.
  studynumbers, idx = np.unique(studynumbers,return_index=True)
  names = names[idx]

  attendees_name = np.append(attendees_name, names)
  attendees_studynumber = np.append(attendees_studynumber, studynumbers)
  dates = np.append(dates, [date]*len(idx))
  activity = np.append(activity, [f'Rådsmøde d. {date}']*len(idx))

#Append the meeting points to this new dataframe.
point = point.append(pd.DataFrame({'Navn' : attendees_name, 'Studienummer' : attendees_studynumber, \
                                   'Aktivitet' : activity, 'Dato' : dates, \
                                   'Kommentar' : ['']*len(dates), 'Point' : [1]*len(dates)}))

#Rows where we need a studynumber.
missing_studynumber = point[point['Studienummer'] == '']

point = point[point['Studienummer'] != '']

point['Studienummer'] = point['Studienummer'].str.lower()

point = point.sort_values('Dato', ascending=False)
def f1(s):
  return max(s, key=len)

point['Dato'] = pd.to_datetime(point['Dato'], format='%d/%m/%Y')
point = point.sort_values(by=['Studienummer','Dato'])

name2studynum = point[['Studienummer','Navn']].groupby(['Studienummer']).agg({'Navn': f1})

leaderboard = pd.merge(name2studynum, point.groupby('Studienummer').sum(), on='Studienummer')
leaderboard = leaderboard.sort_values('Point',ascending=False)


# Top 10 S/M-KID'ere med flest point

In [5]:
leaderboard.iloc[:10]

,Navn,Point
Studienummer,,
s174509,Sofus Albert Høgsbro Rose,20.50
s174474,Matteo Christian Vogt,15.50
s184181,Jonas Bøttzauw Pedersen,9.25
s194325,Jakob Tore Kammeyer Nielsen,9.25
s194269,Kirstine Cort Graae,9.00
s184006,Anne-Marie Kielbasa Jensen,8.75
s194631,Frida Astrup Eriksen,8.50
s174483,Viktor Stenby Johansson,8.50
s180820,Gustav Gamst Larsen,7.50


# Samtlige point for året

In [6]:
pd.set_option('display.max_rows', None)
display(HTML(point.to_html(index=False)))

,Navn,Studienummer,Aktivitet,Dato,Kommentar,Point
,Jonas Olafsson,s144099,Rådsmøde d. 28/09/2020,2020-09-28,,1.00
,Jonas Olafsson,s144099,Gøre klar til møde,2020-09-28,Delte pizza ud inden møde,0.25
,Jonas Olafsson,s144099,Studiecaféudvalget,2020-09-28,,2.00
,Jonas Olafsson,s144099,Økonomiansvarlig,2020-09-28,,3.00
,Aksel Kaastrup Rasmussen,s153569,Rådsmøde d. 14/09/2020,2020-09-14,,1.00
,Emilie Isabella Dahl,s153762,Rådsmøde d. 14/09/2020,2020-09-14,,1.00
,Daniel Jakob Hildebrand,s154410,Rådsmøde d. 14/09/2020,2020-09-14,,1.00
,Daniel Jakob Hildebrand,s154410,Rådsmøde d. 28/09/2020,2020-09-28,,1.00
,Thomas Thorning Røpke,s154430,Rådsmøde d. 14/09/2020,2020-09-14,,1.00
,Toke Bøgelund-Andersen,s164202,Rådsmøde d. 14/09/2020,2020-09-14,,1.00
